In [2]:
import torch

x = torch.tensor(data=[2.0, 3.0], requires_grad = True)
y = x**2
z = 2*y + 3

target = torch.tensor([3.0, 4.0])
loss = torch.sum(torch.abs(z - target))
loss.backward()

print(x.grad, y.grad, z.grad)

tensor([ 8., 12.]) None None


loss.backward()에서 연산 그래프를 쭉 따라가면서 잎 노드 x에 대한 기울기를 계산한다. <br>
여기서 말하는 잎 노드는 다른 별수를 통해 계산되는 y나 z가 아니라 그 자체가 값인 x 같은 노드를 의미한다.<br>
결과적으로 마지막 줄을 실행하면 x.grad는 기울기가 계산되지만 y.grad, z.grad는 잎 노드가 아니기 때문에 결과값이 None이 리턴된다.

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

num_data = 1000
num_epoch = 500

x = init.uniform_(torch.Tensor(num_data, 1), -10, 10)
noise = init.normal_(torch.FloatTensor(num_data, 1), std=1)
y = 2*x+3
y_noise = 2*(x+noise)+3

model = nn.Linear(1,1) #선형회귀모델 호출
                       #우리가 만든 데이터x는 1개의 특성을 가진 데이터 1000개이고
                       #결과y도 1개의 특성을 가진 데이터 1000개이기 때문에 인수로 모델 Linear(1,1)로 생성하였다.
        
loss_func = nn.L1Loss() #L1손실 (차이의 절댓값의 평균)

optimizer = optim.SGD(model.parameters(), lr =0.01)

label = y_noise
for i in range(num_epoch):
    optimizer.zero_grad() #반복할 때마다 지난 반복에서 계산된 기울기를 0으로 초기화한다.
                          #기울기를 초기화해야 새로운 가중치와 편차에 대해서 새로운 기울기를 구할 수 있기 때문이다.
    output = model(x)
    
    loss = loss_func(output, label) #nn.L1Loss(output, label)
    loss.backward() #각 변수에 대한 기울기가 계산된다.
    optimizer.step() #model.parameters()에서 return되는 변수들의 기울기에 학습률을 곱하여 빼줌으로서 업데이트를 한다.
    
    if i % 100 == 0:
        print(loss.data)
param_list = list(model.parameters())
print(param_list[0].item(), param_list[1].item())

tensor(8.7983)
tensor(3.0781)
tensor(2.4785)
tensor(2.0770)
tensor(1.8238)
2.017725944519043 2.400055170059204


init.uniform_(텐서,-10,10)) ::: 텐서의 값을 -10부터 10까지 무작위로 설정한다. <br>
init.normal_(텐서, std=1) ::: 표준편차 1을 가지는 정규분포로 초기화한다.<br>

L1손실 ::: 차이의 절댓값의 평균 $$loss(x,y)={1 \over n}\Sigma|x_i-y_i|$$<br>
데이터 전체를 학습에 한 번 사용하는 주기를 <b>에폭(epoch)</b>라고 한다.